In [ ]:
import cv2
import os
import torch
import gdown
import random
import easydict 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import tensorflow as tf
import torchvision.datasets as dset
from torch.utils import data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import roc_auc_score
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD

In [ ]:
gdown.download('https://drive.google.com/uc?id=15h8EUZOb4qJHy5zE9JDCSCNW28I1xKKG','./images/attack_models.zip',quiet=False)

In [ ]:
os.system("unzip ./images/attack_models.zip -d ./")

In [ ]:
gdown.download('https://drive.google.com/uc?id=1pC3d0nrqB6OtjjN8sKtwI0ebV-RLzIAG','./models/attack_models.zip',quiet=False)

In [ ]:
os.system("unzip ./models/attack_models.zip -d ./")

In [ ]:
gdown.download('https://drive.google.com/uc?id=1BpabkhBO7nMvZaVQN9ONbD4j7kHIa6Dc','./images/test.zip',quiet=False)

In [ ]:
os.system("unzip ./images/test.zip -d ./")

In [ ]:
gdown.download('https://drive.google.com/uc?id=18BBdDEkr_0qSOrIkXDq9ZE95duXrZ4Wc','./images/train.zip',quiet=False)

In [ ]:
os.system("unzip ./images/train.zip -d ./")

In [ ]:
gdown.download('https://drive.google.com/uc?id=1MXVAC2lBrhOcmA7IphIlghsxSCuCBaao','./images/val.zip',quiet=False)

In [ ]:
os.system("unzip ./images/val.zip -d ./")

In [8]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 
Home_directory = '{}'
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [9]:
# Specify the cuda setting if you needed
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ["CUDA_VISIBLE_DEVICES"] = "2"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Generate Detection Classifier

In [26]:
# load train adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/train',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

train_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=True,
                                        num_workers=8)

In [27]:
# load test adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/val',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

val_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=8)

In [28]:
# Specify the cuda setting if you needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = easydict.EasyDict({
        'num_epochs':3,
        'num_epochs_ae':40,
        'lr':1e-3,
        'lr_ae':1e-3,
        'weight_decay':5e-7,
        'weight_decay_ae':5e-3,
        'lr_milestones':[50],
        'batch_size':1024,
        'pretrain':True,
        'latent_dim':32,
        'normal_class':0
                })

In [ ]:
deep_SVDD = TrainerDeepSVDD(args, train_dataloader,device, DeepTaster_forder +'models/classifier/pretrained.pth')
if args.pretrain:
    deep_SVDD.pretrain()
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
net.load_state_dict(state_dict['net_dict'])
c = torch.Tensor(state_dict['center']).to(device)

In [30]:
optimizer = torch.optim.Adam(net.parameters(), lr=args.lr,
                        weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
            milestones=args.lr_milestones, gamma=0.1)


In [ ]:
net.train()
for epoch in range(args.num_epochs):
    total_loss = 0
    for x, _ in train_dataloader:
        x = x.float().to(device)

        optimizer.zero_grad()
        z = net(x)
        loss = torch.mean(torch.sum((z - c) ** 2, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    scheduler.step()
    print('Training Deep SVDD... Epoch: {}, Loss: {:.3f}'.format(
            epoch+1, total_loss/len(train_dataloader)))

In [32]:
scores = []
label = []
net.eval()
with torch.no_grad():
    for x, y in val_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1).cpu()
        for i in range(16):
            if y[i]==0:
                scores.append(score[i])
                label.append(0)
            else:
                scores.append(score[i])
                label.append(1)     

In [33]:
scores.sort()

In [34]:
threshold = float(scores[862])

In [ ]:
print("threshold: ", threshold)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','w')
threshold_file.write(str(threshold))
threshold_file.close()
torch.save(net, DeepTaster_forder +'models/classifier/deepsvdd.pt')

In [ ]:
# Evaluation all attack model

In [36]:
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
c = torch.Tensor(state_dict['center']).to(device)
net=torch.load(DeepTaster_forder +'models/classifier/deepsvdd.pt').to(device)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','r')
threshold=float(threshold_file.readline())
threshold_file.close()

In [37]:
# MAA

In [38]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/test',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [39]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
detection_rate =[]
architecture_list = ["DenseNet", "ResNet","VGG"]
dataset = ["CIFAR10", "MNIST","Tiny-ImageNet","ImageNet"]
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            if y[0]==0:
                detection_rate.append(100 if attack_probability >=144 else 0)
            else:
                detection_rate.append(100 if attack_probability < 144 else 0)
            temp=0
            attack_probability=0

print("MAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Architecture","Attack Prob.","Copy Detection (%)"))
for i in range(3):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Stolen", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))
for i in range(3,12):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("CIFAR10", dataset[int(i/3)], "Benign", architecture_list[i%3],round(result_list[i]/288,2), detection_rate[i] ))


MAA results

Victim          Suspect         Ground Truth    Architecture    Attack Prob.    Copy Detection (%)
CIFAR10         CIFAR10         Stolen          DenseNet        1.0             100            
CIFAR10         CIFAR10         Stolen          ResNet          1.0             100            
CIFAR10         CIFAR10         Stolen          VGG             1.0             100            
CIFAR10         MNIST           Benign          DenseNet        0.0             100            
CIFAR10         MNIST           Benign          ResNet          0.0             100            
CIFAR10         MNIST           Benign          VGG             0.0             100            
CIFAR10         Tiny-ImageNet   Benign          DenseNet        0.05            100            
CIFAR10         Tiny-ImageNet   Benign          ResNet          0.01            100            
CIFAR10         Tiny-ImageNet   Benign          VGG             0.47            100            
CIFAR10         ImageNet

In [40]:
#DAA

In [41]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [42]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [43]:
print(dataset.class_to_idx)

{'DAA_100': 0, 'DAA_20': 1, 'DAA_40': 2, 'DAA_60': 3, 'DAA_80': 4}


In [44]:
print("DAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/DAA/DAA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2), round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


DAA results

Suspect         Ground Truth    Epochs          Model Acc.      Attack Prob.    Copy Detection (%)
CIFAR10         Stolen          20              72.18           0.94            100            
CIFAR10         Stolen          40              73.76           1.0             100            
CIFAR10         Stolen          60              74.03           1.0             100            
CIFAR10         Stolen          80              75.06           1.0             100            
CIFAR10         Stolen          100             74.78           1.0             100            


In [45]:
#SAA

In [46]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/SAA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [47]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [48]:
print("SAA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Victim","Suspect", "Ground Truth","Used Dataset","Attack Prob.","Copy Detection (%)"))
data_percentage = ["10", "30","50","70","90"]
for i in range(5):
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10','CIFAR10', "Stolen", data_percentage[i],round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


SAA results

Victim          Suspect         Ground Truth    Used Dataset    Attack Prob.    Copy Detection (%)
CIFAR10         CIFAR10         Stolen          10              0.31            0              
CIFAR10         CIFAR10         Stolen          30              0.07            0              
CIFAR10         CIFAR10         Stolen          50              0.98            100            
CIFAR10         CIFAR10         Stolen          70              0.88            100            
CIFAR10         CIFAR10         Stolen          90              0.99            100            


In [49]:
#TLA

In [50]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [51]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [52]:
print("TLA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["2","4","6","8"]
for i in range(4):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/TLA/Transfer_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10 to MNIST', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))


TLA results

Suspect            Ground Truth    Epochs          Model Acc.      Attack Prob.    Copy Detection (%)
CIFAR10 to MNIST   Stolen          2               97.8            0.93            100            
CIFAR10 to MNIST   Stolen          4               98.31           0.94            100            
CIFAR10 to MNIST   Stolen          6               98.42           0.97            100            
CIFAR10 to MNIST   Stolen          8               98.56           0.92            100            


In [53]:
#MFA

In [54]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MFA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [55]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [56]:
print("MFA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = [ "500","1000","2500"]
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MFA/Finetuning_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MFA/MNIST_Finetuning_'+data_percentage[i-4]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


MFA results

Suspect         Ground Truth    Used Dataset    Model Acc.      Attack Prob.    Copy Detection (%)
CIFAR10         Stolen          500             74.16           1.0             100            
CIFAR10         Stolen          1000            74.3            0.99            100            
CIFAR10         Stolen          2500            74.57           0.99            100            
MNIST           Benign          500             99.45           0.0             100            
MNIST           Benign          1000            99.45           0.0             100            
MNIST           Benign          2500            99.46           0.0             100            


In [ ]:
#MPA

In [57]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/MPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [58]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [59]:
print("MPA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Pruned (%)","Model Acc.","Attack Prob.","Copy Detection (%)"))
data_percentage = ["20", "40","60"]
for i in range(3,6):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MPA/Prune_'+data_percentage[i-3]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", data_percentage[i-3],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i]  >=144 else 0 ))
for i in range(3):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/MPA/MNIST_Prune_'+data_percentage[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST', "Benign", data_percentage[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] <144 else 0 ))


MPA results

Suspect         Ground Truth    Pruned (%)      Model Acc.      Attack Prob.    Copy Detection (%)
CIFAR10         Stolen          20              70.53           1.0             100            
CIFAR10         Stolen          40              57.96           0.99            100            
CIFAR10         Stolen          60              37.01           1.0             100            
MNIST           Benign          20              98.5            0.05            100            
MNIST           Benign          40              87.47           0.11            100            
MNIST           Benign          60              43.15           0.0             100            


In [ ]:
#DATLA

In [60]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/DATLA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [61]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [62]:
print(dataset.class_to_idx)

{'DATLA_100': 0, 'DATLA_20': 1, 'DATLA_40': 2, 'DATLA_60': 3, 'DATLA_80': 4}


In [63]:
print("DATLA results\n")
print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Epochs","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["100","20","40","60","80"]
for i in [1,2,3,4,0]:
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/DATLA/DATLA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<15} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('CIFAR10', "Stolen", epochs[i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))

DATLA results

Suspect         Ground Truth    Epochs          Model Acc.      Attack Prob.    Copy Detection (%)
CIFAR10         Stolen          20              71.31           1.0             100            
CIFAR10         Stolen          40              72.04           1.0             100            
CIFAR10         Stolen          60              71.53           1.0             100            
CIFAR10         Stolen          80              71.98           1.0             100            
CIFAR10         Stolen          100             71.95           1.0             100            


In [ ]:
#TLPA

In [64]:
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/attack_models/TLPA',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

test_dataloader = torch.utils.data.DataLoader(dataset,
                                          batch_size=16,
                                          shuffle=False,
                                          num_workers=8)

In [65]:
net.eval()
temp=0
test=0
attack_probability=0
result_list = []
with torch.no_grad():
    for x, y in test_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1)
        for i in range(16):
          test+=1
          temp+=1
     
          if score[i]<=threshold:
            attack_probability+=1
          if temp==288:
            result_list.append(attack_probability)
            temp=0
            attack_probability=0

In [67]:
print("TLPA results\n")
print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format("Suspect", "Ground Truth","Used Dataset","Model Acc.","Attack Prob.","Copy Detection (%)"))
epochs = ["10", "30","50","70",'90']
for i in range(5):
    acc_file=open(Home_directory + '/DeepTaster4/models/attack_model/TLPA/TLPA_'+epochs[i]+'.txt','r')
    acc=float(acc_file.readline()[7:13])*100
    acc_file.close()
    print('{:<18} {:<15} {:<15} {:<15} {:<15} {:<15}'.format('MNIST to CIFAR10', "Stolen", epochs [i],round(acc,2),round(result_list[i]/288,2), 100 if result_list[i] >=144 else 0 ))

TLPA results

Suspect            Ground Truth    Used Dataset    Model Acc.      Attack Prob.    Copy Detection (%)
MNIST to CIFAR10   Stolen          10              98.54           1.0             100            
MNIST to CIFAR10   Stolen          30              98.14           1.0             100            
MNIST to CIFAR10   Stolen          50              98.13           1.0             100            
MNIST to CIFAR10   Stolen          70              98.17           1.0             100            
MNIST to CIFAR10   Stolen          90              98.59           1.0             100            
